In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('D:/Py/Adidas Sales Dashboard/Adidas.xlsx')
df.head()

,Retailer,RetailerID,InvoiceDate,Region,State,City,Product,PriceperUnit,UnitsSold,TotalSales,OperatingProfit,OperatingMargin,SalesMethod
0,Foot Locker,1185732,2020-02-03,Northeast,New York,New York,Women's Athletic Footwear,45.0,825,371250.0,129937.5,0.35,Outlet
1,Foot Locker,1185732,2020-02-04,Northeast,New York,New York,Men's Apparel,60.0,825,495000.0,148500.0,0.30,Outlet
2,Foot Locker,1185732,2020-02-05,Northeast,New York,New York,Women's Apparel,50.0,950,475000.0,118750.0,0.25,Outlet
3,Foot Locker,1185732,2020-02-06,Northeast,New York,New York,Men's Street Footwear,60.0,1220,732000.0,366000.0,0.50,Outlet
4,Foot Locker,1185732,2020-02-07,Northeast,New York,New York,Men's Athletic Footwear,55.0,925,508750.0,152625.0,0.30,Outlet


In [3]:
df.columns

Index(['Retailer', 'RetailerID', 'InvoiceDate', 'Region', 'State', 'City',
       'Product', 'PriceperUnit', 'UnitsSold', 'TotalSales', 'OperatingProfit',
       'OperatingMargin', 'SalesMethod'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Retailer         9648 non-null   object        
 1   RetailerID       9648 non-null   int64         
 2   InvoiceDate      9648 non-null   datetime64[ns]
 3   Region           9648 non-null   object        
 4   State            9648 non-null   object        
 5   City             9648 non-null   object        
 6   Product          9648 non-null   object        
 7   PriceperUnit     9648 non-null   float64       
 8   UnitsSold        9648 non-null   int64         
 9   TotalSales       9648 non-null   float64       
 10  OperatingProfit  9648 non-null   float64       
 11  OperatingMargin  9648 non-null   float64       
 12  SalesMethod      9648 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(6)
memory usage: 980.0+ KB


In [14]:
df.describe()

,RetailerID,InvoiceDate,PriceperUnit,UnitsSold,TotalSales,OperatingProfit,OperatingMargin
count,9.648000e+03,9648,9648.000000,9648.000000,9648.000000,9648.000000,9648.000000
mean,1.173850e+06,2021-05-10 15:20:44.776119296,45.216625,256.930037,93250.893968,34408.018392,0.422991
min,1.128299e+06,2020-01-01 00:00:00,7.000000,0.000000,0.000000,0.000000,0.100000
25%,1.185732e+06,2021-02-17 00:00:00,35.000000,106.000000,4254.500000,1921.752500,0.350000
50%,1.185732e+06,2021-06-04 00:00:00,45.000000,176.000000,9576.000000,4371.420000,0.410000
75%,1.185732e+06,2021-09-16 00:00:00,55.000000,350.000000,150000.000000,52062.500000,0.490000
max,1.197831e+06,2021-12-31 00:00:00,110.000000,1275.000000,825000.000000,390000.000000,0.800000
std,2.636038e+04,NaN,14.705397,214.252030,141852.775230,54135.070618,0.097197


In [16]:
df.isnull().sum()

Retailer           0
RetailerID         0
InvoiceDate        0
Region             0
State              0
City               0
Product            0
PriceperUnit       0
UnitsSold          0
TotalSales         0
OperatingProfit    0
OperatingMargin    0
SalesMethod        0
InvoiceMonth       0
dtype: int64

### 1. Total Sales by Region

In [5]:
import plotly.express as px

region_sales = df.groupby('Region')['TotalSales'].sum().reset_index()
fig = px.bar(region_sales, x='Region', y='TotalSales', title='Total Sales by Region',
             labels={'TotalSales':'Total Sales ($)'})
fig 

### 2. Sales Trend Over Time


In [6]:
import plotly.graph_objects as go

df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')
sales_trend = df.groupby('InvoiceMonth')['TotalSales'].sum().reset_index()
fig = go.Figure()
fig.add_trace(go.Scatter(x=sales_trend['InvoiceMonth'].astype(str), y=sales_trend['TotalSales'],
                         mode='lines', name='Sales Trend'))
fig.update_layout(title='Sales Trend Over Time', xaxis_title='Month', yaxis_title='Total Sales ($)')
fig.show()

### 3. Top Products by Sales

In [7]:
top_products = df.groupby('Product')['TotalSales'].sum().sort_values(ascending=False).head(10).reset_index()
fig = px.bar(top_products, x='Product', y='TotalSales', title='Top 10 Products by Sales',
             labels={'TotalSales':'Total Sales ($)'})
fig.show()

### 4. Sales Method Distribution


In [8]:
sales_method = df.groupby('SalesMethod')['TotalSales'].sum().reset_index()
fig = px.pie(sales_method, names='SalesMethod', values='TotalSales', title='Sales Method Distribution')
fig.show()

### 5. Operating Profit by Region

In [9]:
region_profit = df.groupby('Region')['OperatingProfit'].sum().reset_index()
fig = px.bar(region_profit, x='Region', y='OperatingProfit', title='Operating Profit by Region',
             labels={'OperatingProfit':'Operating Profit ($)'})
fig.show()

### 6. Profitability Bubble Chart (by Product)


In [10]:
product_profit = df.groupby('Product').agg({'TotalSales':'sum', 'OperatingProfit':'sum'}).reset_index()
fig = px.scatter(product_profit, x='TotalSales', y='OperatingProfit', size='OperatingProfit', 
                 color='Product', title='Profitability by Product', labels={'TotalSales':'Total Sales ($)', 
                                                                            'OperatingProfit':'Operating Profit ($)'})
fig.show()

### 7. Map of Total Sales by State

In [13]:
# Treemap: Total Sales by Region and City
treemap_data = df.groupby(["Region", "City"])["TotalSales"].sum().reset_index()
treemap_fig = px.treemap(treemap_data, path=["Region", "City"], values="TotalSales",
                         color="City", title="Sales by Region and City",
                         hover_data=["TotalSales"], template="seaborn")
treemap_fig.show()

### 8. Units Sold by City

In [12]:
city_sales = df.groupby('City')['UnitsSold'].sum().reset_index()
fig = px.bar(city_sales, x='City', y='UnitsSold', title='Units Sold by City',
             labels={'UnitsSold':'Units Sold'})
fig.show()

### 9. Operating Margin by Sales Method


In [13]:
method_margin = df.groupby('SalesMethod')['OperatingMargin'].mean().reset_index()
fig = px.bar(method_margin, x='SalesMethod', y='OperatingMargin', title='Operating Margin by Sales Method',
             labels={'OperatingMargin':'Operating Margin (%)'})
fig.show()